In [ ]:
import scanpy as sc
import numpy as np
from tqdm.notebook import tqdm
import scipy.stats as stats
import pandas as pd
import json
import os
import seaborn as sns
import matplotlib.pyplot as plt
import shapely
from sklearn.neighbors import NearestNeighbors
from PIL import Image, ImageDraw
import glob

Put in the path to the SI adatas that we want the crypt-villus axis calculated on

In [ ]:
path_adata = r"/mnt/sata1/Analysis_Alex/uninfected"
adata = sc.read(os.path.join(path_adata, "final_celltyped_and_axes.h5ad"))

In [ ]:
def create_base_image_type(adata, downsize=20):

    points = adata.obsm["X_spatial"] / downsize
    types = adata.obs["topic"].values

    # Define the size of the image (adjust as needed)
    image_width = 2000
    image_height = 2000

    # Create a white canvas as the base image
    base_image = Image.new("RGB", (image_width, image_height), (255, 255, 255))

    # Draw the points on the image
    draw = ImageDraw.Draw(base_image)
    point_size = 1  # Size of the points
    ct = 0
    for point in points:
        draw.ellipse(
            (
                point[0] - point_size,
                point[1] - point_size,
                point[0] + point_size,
                point[1] + point_size,
            ),
            fill=adata.uns["topic_colors"][
                np.where(adata.obs["topic"].cat.categories == types[ct])[0]
            ][0],
            outline=adata.uns["topic_colors"][
                np.where(adata.obs["topic"].cat.categories == types[ct])[0]
            ][0],
            width=0,
        )
        ct += 1

    return base_image

In [ ]:
try:
    os.mkdir(os.path.join(path_adata, "recalculating_axes"))
except:
    pass

In [ ]:
sc.pl.embedding(adata, basis="spatial", color="topic")

In [ ]:
for batch in np.unique(adata.obs["batch"]):
    try:
        adata_copy = adata[adata.obs["batch"] == batch].copy()
        all_spatial = adata_copy.obsm["X_spatial"]

        try:
            adata_copy.X = adata_copy.X.A
        except:
            print("Adata already in array format")

        try:
            os.mkdir(os.path.join(path_adata, "recalculating_axes", batch))
        except:
            pass

        downsize = 4
        base_image = create_base_image_type(adata_copy, downsize=downsize)
        plt.imshow(base_image)
        plt.show()
        file_path = os.path.join(
            path_adata, "recalculating_axes", batch, "label_img.png"
        )
        base_image.save(file_path)
        adata_copy.uns["unrolling_downsize"] = downsize
        adata_copy.write(
            os.path.join(
                path_adata, "recalculating_axes", batch, "03_intial_neighborhoods.h5ad"
            )
        )
    except:
        print(batch + " not generated yet")